In [1]:
import numpy as np
import pandas as pd

In [224]:
#日期，业务种类，重量，揽投机构，道段(揽收只有业务员信息，没有道段故不收入)

#读取揽收数据
data_lanshou = pd.read_csv('江门揽收数据.csv', encoding='gb18030', usecols=[1,3,6,8])
data_lanshou = data_lanshou[['D_CLCT_DATE','V_BUSI_TYPE', 'F_WEIGH_WEIGHT', 'V_CLCT_ORG_CODE']]#调整一下顺序
data_lanshou['D_CLCT_DATE'] = pd.to_datetime(data_lanshou['D_CLCT_DATE'])
#加入经纬度
data_lanshou_ = pd.read_csv('coord/lanshou.csv', encoding='gb18030', usecols=[1,2])
data_lanshou['lng'] = data_lanshou_['lng']
data_lanshou['lat'] = data_lanshou_['lat']

#读取投递数据1
data_toudi21 = pd.read_csv('toudi_21.csv', encoding='gb18030', usecols=[0,2,3,4,7])
data_toudi21['道段'] = data_toudi21['道段'].str.split(' ', expand=True)[0]
data_toudi21['投递日期'] = pd.to_datetime(data_toudi21['投递日期'])
#加入经纬度
data_toudi21_  = pd.read_csv('coord/toudi_21.csv', encoding='gb18030', usecols=[1,2])
data_toudi21['lng'] = data_toudi21_['lng']
data_toudi21['lat'] = data_toudi21_['lat']

#读取投递数据2
data_toudi22 = pd.read_csv('toudi_21.csv', encoding='gb18030', usecols=[0,2,3,4,7])
data_toudi22['道段'] = data_toudi22['道段'].str.split(' ', expand=True)[0]
data_toudi22['投递日期'] = pd.to_datetime(data_toudi22['投递日期'])
#加入经纬度
data_toudi22_  = pd.read_csv('coord/toudi_21.csv', encoding='gb18030', usecols=[1,2])
data_toudi22['lng'] = data_toudi22_['lng']
data_toudi22['lat'] = data_toudi22_['lat']

#读取投递数据3
data_toudi206 = pd.read_csv('toudi_21.csv', encoding='gb18030', usecols=[0,2,3,4,7])
data_toudi206['道段'] = data_toudi206['道段'].str.split(' ', expand=True)[0]
data_toudi206['投递日期'] = pd.to_datetime(data_toudi206['投递日期'])
#加入经纬度
data_toudi206_  = pd.read_csv('coord/toudi_21.csv', encoding='gb18030', usecols=[1,2])
data_toudi206['lng'] = data_toudi206_['lng']
data_toudi206['lat'] = data_toudi206_['lat']

In [259]:
#exclude为不需要的日期列表，下述代码将周末的数据排除
exclude = ["2020-08-01","2020-08-02","2020-08-08","2020-08-09","2020-08-15","2020-08-16","2020-08-22","2020-08-23","2020-08-29","2020-08-30"]

bools = []
for row in data_lanshou.values:
    bools.append(str(row[0].date()) not in exclude)
data_lanshou = data_lanshou.loc[bools]

bools = []
for row in data_toudi21.values:
    bools.append(str(row[0].date()) not in exclude)
data_toudi21 = data_toudi21.loc[bools]

bools = []
for row in data_toudi22.values:
    bools.append(str(row[0].date()) not in exclude)
data_toudi22 = data_toudi22.loc[bools]
    
bools = []
for row in data_toudi206.values:
    bools.append(str(row[0].date()) not in exclude)
data_toudi206 = data_toudi206.loc[bools]

In [260]:
#揽收默认的列属性不太一样，改成与投递一致
new_cols = ["投递日期","业务种类","重量","投递机构","lng","lat"]
data_lanshou.columns = new_cols

In [286]:
#按部门分开存储，放置于列表中
lanshou = []
toudi21 = []
toudi22 = []
toudi206 = []

departments = pd.read_excel('江门补充资料/江门机构.xlsx', usecols=[3])
for department in departments.values:
    batch = data_lanshou.loc[data_lanshou['投递机构']==int(department)]
    lanshou.append(batch)
    batch = data_toudi21.loc[data_toudi21['投递机构']==int(department)]
    toudi21.append(batch)
    batch = data_toudi22.loc[data_toudi22['投递机构']==int(department)]
    toudi22.append(batch)
    batch = data_toudi206.loc[data_toudi206['投递机构']==int(department)]
    toudi206.append(batch)

In [287]:
#揽收数据没有道段，故不划分道段

#对toudi21的每个机构进行处理
for department in toudi21:
    department_temp = department.loc[department['lng'].isnull().values.tolist()] #不含经纬度的数据
    department_done = department.drop(department_temp.index)
    roads = department_temp['道段'].unique()
    roads = roads[~pd.isnull(roads)]
    #对某个机构中的每个道段进行处理
    for road in roads:
        data_road = department_done.loc[department_done['道段']==road]
        maxy = data_road['lng'].max()
        miny = data_road['lng'].min()
        maxx = data_road['lat'].max()
        minx = data_road['lat'].min()
        temp_road = department_temp.loc[department_temp['道段']==road]
        lng = np.random.random(len(temp_road)) * (maxy - miny) + miny
        lat = np.random.random(len(temp_road)) * (maxx - minx) + minx
        department_temp.loc[department_temp['道段']==road, 'lng'] = lng
        department_temp.loc[department_temp['道段']==road, 'lat'] = lat
    #对没有道段数据的记录，即nan值做特殊处理(取处理的最后一个道段的随机经纬度)
    nan = department_temp.loc[department_temp['道段'].isnull().values.tolist()]
    if len(nan) > 0:
        lng = np.random.random(len(nan)) * (maxy - miny) + miny
        lat = np.random.random(len(nan)) * (maxx - minx) + minx
        department_temp.loc[nan.index, 'lng'] = lng
        department_temp.loc[nan.index, 'lat'] = lat
    
    department.loc[department['lng'].isnull().values.tolist(), 'lng'] = department_temp['lng']
    department.loc[department['lat'].isnull().values.tolist(), 'lat'] = department_temp['lat']

In [288]:
#对toudi22的每个机构进行处理
for department in toudi22:
    department_temp = department.loc[department['lng'].isnull().values.tolist()] #不含经纬度的数据
    department_done = department.drop(department_temp.index)
    roads = department_temp['道段'].unique()
    roads = roads[~pd.isnull(roads)]
    #对某个机构中的每个道段进行处理
    for road in roads:
        data_road = department_done.loc[department_done['道段']==road]
        maxy = data_road['lng'].max()
        miny = data_road['lng'].min()
        maxx = data_road['lat'].max()
        minx = data_road['lat'].min()
        temp_road = department_temp.loc[department_temp['道段']==road]
        lng = np.random.random(len(temp_road)) * (maxy - miny) + miny
        lat = np.random.random(len(temp_road)) * (maxx - minx) + minx
        department_temp.loc[department_temp['道段']==road, 'lng'] = lng
        department_temp.loc[department_temp['道段']==road, 'lat'] = lat
    #对没有道段数据的记录，即nan值做特殊处理(取处理的最后一个道段的随机经纬度)
    nan = department_temp.loc[department_temp['道段'].isnull().values.tolist()]
    if len(nan) > 0:
        lng = np.random.random(len(nan)) * (maxy - miny) + miny
        lat = np.random.random(len(nan)) * (maxx - minx) + minx
        department_temp.loc[nan.index, 'lng'] = lng
        department_temp.loc[nan.index, 'lat'] = lat
    
    department.loc[department['lng'].isnull().values.tolist(), 'lng'] = department_temp['lng']
    department.loc[department['lat'].isnull().values.tolist(), 'lat'] = department_temp['lat']

In [289]:
#对toudi206的每个机构进行处理
for department in toudi206:
    department_temp = department.loc[department['lng'].isnull().values.tolist()] #不含经纬度的数据
    department_done = department.drop(department_temp.index)
    roads = department_temp['道段'].unique()
    roads = roads[~pd.isnull(roads)]
    #对某个机构中的每个道段进行处理
    for road in roads:
        data_road = department_done.loc[department_done['道段']==road]
        maxy = data_road['lng'].max()
        miny = data_road['lng'].min()
        maxx = data_road['lat'].max()
        minx = data_road['lat'].min()
        temp_road = department_temp.loc[department_temp['道段']==road]
        lng = np.random.random(len(temp_road)) * (maxy - miny) + miny
        lat = np.random.random(len(temp_road)) * (maxx - minx) + minx
        department_temp.loc[department_temp['道段']==road, 'lng'] = lng
        department_temp.loc[department_temp['道段']==road, 'lat'] = lat
    #对没有道段数据的记录，即nan值做特殊处理(取处理的最后一个道段的随机经纬度)
    nan = department_temp.loc[department_temp['道段'].isnull().values.tolist()]
    if len(nan) > 0:
        lng = np.random.random(len(nan)) * (maxy - miny) + miny
        lat = np.random.random(len(nan)) * (maxx - minx) + minx
        department_temp.loc[nan.index, 'lng'] = lng
        department_temp.loc[nan.index, 'lat'] = lat
    
    department.loc[department['lng'].isnull().values.tolist(), 'lng'] = department_temp['lng']
    department.loc[department['lat'].isnull().values.tolist(), 'lat'] = department_temp['lat']

In [294]:
for department in toudi21:
    roads = department_temp['道段'].unique()
    #对某个机构中的每个道段进行处理
    for road in roads:
        print(road)
        data_road = department.loc[department['道段']==road]
        maxy = data_road['lng'].max()
        miny = data_road['lng'].min()
        maxx = data_road['lat'].max()
        minx = data_road['lat'].min()
        print(maxy,miny,maxx,minx)

In [307]:
data_road = toudi21[0].loc[toudi21[0]['道段']=="02",:]
maxy = data_road['lng'].max()
miny = data_road['lng'].min()
maxx = data_road['lat'].max()
minx = data_road['lat'].min()
print(maxy,miny,maxx,minx)

121.26944 102.264449 35.020459 18.701982


In [308]:
toudi21[0].loc[toudi21[0]['lng'] == 121.26944]

,投递日期,业务种类,重量,投递机构,道段,lng,lat
375813,2020-08-12 16:56:56,法院专递,0.3,52900009,02,121.26944,31.247288
